In [1]:
import tensorflow as tf
from tensorflow import keras
import time
import numpy as np

2023-07-13 21:25:25.834975: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-13 21:25:39.123383: E itex/core/kernels/xpu_kernel.cc:38] XPU-GPU kernel not supported.
If you need help, create an issue at https://github.com/intel/intel-extension-for-tensorflow/issues


<h3>Loading the Fashion MNIST Test dataset from Tensorflow-Keras</h3>

In [2]:
(_, _), (test_images, test_labels) = keras.datasets.fashion_mnist.load_data()

test_images = test_images.reshape((-1, 28, 28, 1))
test_images = test_images.astype("float32")/255

<h3>Optimising the model to IR format with OpenVino Model Optimiser</h3>

* Output is saved in models directory as IR_model

In [45]:
!mo --saved_model_dir ../models/itex_saved_model --output_dir ../models/IR_model

2023-07-13 21:28:39.165368: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-13 21:28:43.790642: E itex/core/kernels/xpu_kernel.cc:38] XPU-GPU kernel not supported.
If you need help, create an issue at https://github.com/intel/intel-extension-for-tensorflow/issues
[ INFO ] The model was converted to IR v11, the latest model format that corresponds to the source DL framework input/output format. While IR v11 is backwards compatible with OpenVINO Inference Engine API v1.0, please use API v2.0 (as of 2022.1) to take advantage of the latest improvements in IR v11.
Find more information about API v2.0 and IR v11 at https://docs.openvino.ai/2023.0/openvino_2_0_transition_guide.html
[ INFO ] IR generated by new TensorFlow Frontend is compatible o

<h3>Importing the Openvino Runtime and loading the IR Model</h3>

In [46]:
import openvino.runtime as ov

core = ov.Core()

compiled_model = core.compile_model("../models/IR_model/saved_model.xml", "AUTO")

In [47]:
infer_request = compiled_model.create_infer_request()

In [48]:
image_array = np.array(test_images)

image_array = image_array.astype(np.float32)


<h3>Input tensor is defined and set</h3>

In [49]:
input_tensor = ov.Tensor(array=image_array, shared_memory=True)

infer_request.set_input_tensor(input_tensor)

<h3>Output is predicted using Openvino Optimised model with Openvino Runtime API and Inferrence time is noted</h3>

In [50]:
start_time = time.time()
output = infer_request.infer(input_tensor)
end_time = time.time()

In [51]:
inference_time = end_time - start_time
print(f"OpenVino inference time : {inference_time}")

OpenVino inference time : 1.6626226902008057


<h3>Output is predicted using normal model trained with Intel-tensorflow-extension and inferrence time is noted</h3>

In [52]:
model = tf.keras.models.load_model('../models/itex_model.h5')

In [53]:
start_time = time.time()
model.predict(test_images)
end_time = time.time()

  1/313 [..............................] - ETA: 48s

2023-07-13 21:28:59.003423: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type CPU is enabled.
2023-07-13 21:28:59.010291: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type CPU is enabled.
2023-07-13 21:28:59.020625: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type CPU is enabled.
2023-07-13 21:28:59.081413: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type CPU is enabled.


313/313 [==============================] - 3s 10ms/step


In [54]:
inference_time = end_time - start_time
print(inference_time)

5.283219814300537


<h2>Inference time</h2>

* with Openvino: 1.5 sec
* without Openvino: 5.2 sec
<h1>~70% reduction in Inferrence time is observed with OpenVino optimisation</h1>